In [1]:
%config Completer.use_jedi = False

In [2]:
import glob
import pandas as pd
import numpy as np
import IPython.display as ipd #palying audio
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import os

SR = 16000

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

In [3]:
csv_paths = glob.glob(f'data_features/*.csv')

In [4]:
df_list = []
for path in csv_paths:
    df_list.append(pd.read_csv(path, index_col=0))

In [5]:
df = pd.concat(df_list)

In [6]:
df.shape

(50654, 192)

In [7]:
df.head()

,wave_mean,wave_median,wave_std,wave_min,wave_max,wave_amp,wave_SNR,wave_kurtosis,wave_skew,wave_percentile_10,...,fft_median,fft_std,fft_min,fft_max,fft_amp,fft_SNR,fft_entropy,fft_kurtosis,fft_skew,path
0,-0.000003,-0.000031,0.010094,-0.039036,0.042614,0.081650,-0.000334,-0.045107,0.006721,-0.012905,...,0.598390,3.725990,0.000165,264.446513,264.446349,0.417466,11.032971,841.570713,17.925001,data/-6_dB_slider/slider/id_04/normal/00000003...
1,0.000006,0.000011,0.008588,-0.039116,0.037731,0.076847,0.000728,-0.019601,-0.009085,-0.011028,...,1.012292,2.991213,0.001762,154.644755,154.642993,0.564819,11.386249,353.281951,12.704317,data/-6_dB_slider/slider/id_04/normal/00000013...
2,-0.000003,-0.000013,0.009915,-0.039959,0.042530,0.082489,-0.000321,0.006090,-0.003327,-0.012714,...,0.653014,3.607357,0.002463,369.202087,369.199624,0.456992,11.090701,1460.712269,19.849537,data/-6_dB_slider/slider/id_04/normal/00000179...
3,0.000002,0.000011,0.008567,-0.041214,0.044659,0.085873,0.000188,0.007006,-0.002908,-0.011013,...,0.634151,3.129056,0.004116,179.793890,179.789774,0.446533,11.154298,541.783872,15.340489,data/-6_dB_slider/slider/id_04/normal/00000235...
4,0.000002,-0.000042,0.010389,-0.044312,0.045147,0.089458,0.000182,-0.065827,0.010113,-0.013371,...,0.632426,3.949066,0.001332,370.385849,370.384518,0.327632,11.101559,2531.646995,36.889158,data/-6_dB_slider/slider/id_04/normal/00000186...


# Scaling

In [8]:
cols_scaling = df.drop('path', axis=1).columns

In [9]:
scaler = StandardScaler()

In [10]:
df[cols_scaling] = scaler.fit_transform(df[cols_scaling])

In [11]:
df.head()

,wave_mean,wave_median,wave_std,wave_min,wave_max,wave_amp,wave_SNR,wave_kurtosis,wave_skew,wave_percentile_10,...,fft_median,fft_std,fft_min,fft_max,fft_amp,fft_SNR,fft_entropy,fft_kurtosis,fft_skew,path
0,-1.290201,-0.468159,1.289060,0.018168,0.047237,0.017450,-0.753597,-0.258452,-0.134690,-1.282945,...,0.630646,1.265004,-1.013547,0.570983,0.570991,-0.294127,-0.159342,-0.257215,-0.193474,data/-6_dB_slider/slider/id_04/normal/00000003...
1,3.042385,-0.012988,0.714383,0.013820,-0.179098,-0.105019,2.461824,-0.256425,-0.230135,-0.737195,...,2.449424,0.535673,0.570091,-0.318938,-0.318943,0.690914,1.024023,-0.447552,-0.524359,data/-6_dB_slider/slider/id_04/normal/00000013...
2,-1.204482,-0.281953,1.220891,-0.031936,0.043347,0.038851,-0.713682,-0.254383,-0.195361,-1.227483,...,0.870676,1.147250,1.265085,1.420005,1.419995,-0.029898,0.034034,-0.015872,-0.071498,data/-6_dB_slider/slider/id_04/normal/00000179...
3,0.952128,-0.012988,0.706248,-0.100052,0.142015,0.125134,0.826371,-0.254310,-0.192833,-0.732758,...,0.787788,0.672494,2.904106,-0.115110,-0.115133,-0.099814,0.247063,-0.374073,-0.357279,data/-6_dB_slider/slider/id_04/normal/00000235...
4,1.080010,-0.592297,1.401635,-0.268168,0.164649,0.216572,0.809526,-0.260099,-0.114207,-1.418384,...,0.780210,1.486426,0.143228,1.429599,1.429598,-0.894657,0.070407,0.401582,1.008467,data/-6_dB_slider/slider/id_04/normal/00000186...


In [12]:
df.loc[:, 'machine'] = df['path'].apply(lambda string: string.split('/')[2])
df.loc[:, 'abnormal'] = df['path'].apply(lambda string: 1 if 'abnormal' in string else 0)
df.loc[:, 'id'] = df['path'].apply(lambda string: string.split('/')[3][3:])

In [13]:
df.groupby(['id', 'machine']).count()

wave_mean  wave_median  wave_std  wave_min  wave_max  wave_amp  \
id machine                                                                   
00 fan           4254         4254      4254      4254      4254      4254   
   pump          3447         3447      3447      3447      3447      3447   
   slider        4272         4272      4272      4272      4272      4272   
   valve         3330         3330      3330      3330      3330      3330   
02 fan           4125         4125      4125      4125      4125      4125   
   pump          3348         3348      3348      3348      3348      3348   
   slider        4005         4005      4005      4005      4005      4005   
   valve         2484         2484      2484      2484      2484      2484   
04 fan           3741         3741      3741      3741      3741      3741   
   pump          2406         2406      2406      2406      2406      2406   
   slider        2136         2136      2136      2136      2136      2136   
   valve         3360         3360      3360      3360      3360      3360   
06 fan           2752         2752      2752      2752      2752      2752   
   pump          2622         2622      2622      2622      2622      2622   
   slider        1299         1299      1299      1299      1299      1299   
   valve         3073         3073      3073      3073      3073      3073   

            wave_SNR  wave_kurtosis  wave_skew  wave_percentile_10  ...  \
id machine                                                          ...   
00 fan          4254           4254       4254                4254  ...   
   pump         3447           3447       3447                3447  ...   
   slider       4272           4272       4272                4272  ...   
   valve        3330           3330       3330                3330  ...   
02 fan          4125           4125       4125                4125  ...   
   pump         3348           3348       3348                3348  ...   
   slider       4005           4005       4005                4005  ...   
   valve        2484           2484       2484                2484  ...   
04 fan          3741           3741       3741                3741  ...   
   pump         2406           2406       2406                2406  ...   
   slider       2136           2136       2136                2136  ...   
   valve        3360           3360       3360                3360  ...   
06 fan          2752           2752       2752                2752  ...   
   pump         2622           2622       2622                2622  ...   
   slider       1299           1299       1299                1299  ...   
   valve        3073           3073       3073                3073  ...   

            fft_std  fft_min  fft_max  fft_amp  fft_SNR  fft_entropy  \
id machine                                                             
00 fan         4254     4254     4254     4254     4254         4254   
   pump        3447     3447     3447     3447     3447         3447   
   slider      4272     4272     4272     4272     4272         4272   
   valve       3330     3330     3330     3330     3330         3330   
02 fan         4125     4125     4125     4125     4125         4125   
   pump        3348     3348     3348     3348     3348         3348   
   slider      4005     4005     4005     4005     4005         4005   
   valve       2484     2484     2484     2484     2484         2484   
04 fan         3741     3741     3741     3741     3741         3741   
   pump        2406     2406     2406     2406     2406         2406   
   slider      2136     2136     2136     2136     2136         2136   
   valve       3360     3360     3360     3360     3360         3360   
06 fan         2752     2752     2752     2752     2752         2752   
   pump        2622     2622     2622     2622     2622         2622   
   slider      1299     1299     1299     1299     1299         1299   
   valve       3073     3073    

In [14]:
dummies = pd.get_dummies(df['machine'], prefix='Machine', drop_first=True)

In [15]:
df = df.join(dummies)

In [16]:
df.drop('machine', axis=1, inplace=True)

In [ ]:
# Split train/test and validation sets

In [ ]:
drop_list = ['path']

In [ ]:
validation = df[df['id'] == '06'].drop(drop_list, axis=1)
train_test = df[df['id'] != '06'].drop(drop_list, axis=1)

In [ ]:
df.head()

In [ ]:
train_test.shape

In [ ]:
print((df == -np.inf).any().any())
print((df == np.inf).any().any())

In [ ]:
model_diccie = {'rfc': RandomForestClassifier(verbose=1, n_jobs=-1),
                'svm': SVC(verbose=1, probability=True),
                'knn': KNeighborsClassifier(),
                'tree': DecisionTreeClassifier(),
               'lr': LogisticRegression(verbose=1,)}

# Split by id, 1/10th of data

In [ ]:
model_scores = {}
plt.figure(figsize=(14, 14))
plt.grid()
for model_key in model_diccie.keys():
    print(model_key)
    model_scores[model_key] = {}
    X_train = train_test[train_test['id'] != '04'].drop('abnormal', axis=1)[::10]
    X_test = train_test[train_test['id'] == '04'].drop('abnormal', axis=1)

    y_train = train_test[train_test['id'] != '04']['abnormal'][::10]
    y_test = train_test[train_test['id'] == '04']['abnormal']
    model = model_diccie[model_key]
    model.fit(X=X_train, y=y_train)
    probs = model.predict_proba(X_test)[:, 1]
    model_scores[model_key] = {'probs': probs, 'y_test': y_test}
    lr_auc = roc_auc_score(y_test, probs)
    print(model_key + ': ROC AUC=%.3f' % (lr_auc))

    # calculate roc curves
    lr_fpr, lr_tpr, _ = roc_curve(y_test, probs)
    # plot the roc curve for the model
    
    pyplot.plot(lr_fpr, lr_tpr, marker='.', label=model_key)
    # axis labels
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    # show the legend
    pyplot.legend()
    # show the plot
pyplot.show()

In [ ]:
model_scores = {}
plt.figure(figsize=(14, 14))
plt.grid()
for model_key in model_diccie.keys():
    print(model_key)
    model_scores[model_key] = {}
    X_train = train_test[train_test['id'] != '04'].drop('abnormal', axis=1)[::5]
    X_test = train_test[train_test['id'] == '04'].drop('abnormal', axis=1)

    y_train = train_test[train_test['id'] != '04']['abnormal'][::5]
    y_test = train_test[train_test['id'] == '04']['abnormal']
    model = model_diccie[model_key]
    model.fit(X=X_train, y=y_train)
    probs = model.predict_proba(X_test)[:, 1]
    model_scores[model_key] = {'probs': probs, 'y_test': y_test}
    lr_auc = roc_auc_score(y_test, probs)
    print(model_key + ': ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    lr_fpr, lr_tpr, _ = roc_curve(y_test, probs)
    # plot the roc curve for the model
    pyplot.plot(lr_fpr, lr_tpr, marker='.', label=model_key)
    # axis labels
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    # show the legend
    pyplot.legend()
    # show the plot
pyplot.show()

# biased

In [ ]:
model_diccie = {'rfc': RandomForestClassifier(verbose=1, n_jobs=-1),
                #'svm': SVC(verbose=1, probability=True),
                'knn': KNeighborsClassifier(),
                'tree': DecisionTreeClassifier(),
               'lr': LogisticRegression(verbose=1)}

In [ ]:
model_scores = {}
plt.figure(figsize=(14, 14))
plt.grid()
for model_key in model_diccie.keys():
    print(model_key)
    model_scores[model_key] = {}
    X_train = train_test.drop('abnormal', axis=1)[::2]
    X_test = train_test.drop('abnormal', axis=1)[1::2]

    y_train = train_test['abnormal'][::2]
    y_test = train_test['abnormal'][1::2]
    model = model_diccie[model_key]
    model.fit(X=X_train, y=y_train)
    probs = model.predict_proba(X_test)[:, 1]
    model_scores[model_key] = {'probs': probs, 'y_test': y_test}
    lr_auc = roc_auc_score(y_test, probs)
    print(model_key + ': ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    lr_fpr, lr_tpr, _ = roc_curve(y_test, probs)
    # plot the roc curve for the model
    pyplot.plot(lr_fpr, lr_tpr, marker='.', label=model_key)
    # axis labels
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    # show the legend
    pyplot.legend()
    # show the plot
pyplot.show()

In [ ]:
model_diccie = {'rfc': RandomForestClassifier(verbose=1, n_jobs=-1),
                #'svm': SVC(verbose=1, probability=True),
                'knn': KNeighborsClassifier(),
                'tree': DecisionTreeClassifier(),
               'lr': LogisticRegression(verbose=1)}

In [ ]:
model_scores = {}
plt.figure(figsize=(14, 14))
plt.grid()
for model_key in model_diccie.keys():
    print(model_key)
    model_scores[model_key] = {}
    X_train = train_test[train_test['id'] != '04'].drop('abnormal', axis=1)[::1]
    X_test = train_test[train_test['id'] == '04'].drop('abnormal', axis=1)

    y_train = train_test[train_test['id'] != '04']['abnormal'][::1]
    y_test = train_test[train_test['id'] == '04']['abnormal']
    model = model_diccie[model_key]
    model.fit(X=X_train, y=y_train)
    probs = model.predict_proba(X_test)[:, 1]
    model_scores[model_key] = {'probs': probs, 'y_test': y_test}
    lr_auc = roc_auc_score(y_test, probs)
    print(model_key + ': ROC AUC=%.3f' % (lr_auc))

    # calculate roc curves
    lr_fpr, lr_tpr, _ = roc_curve(y_test, probs)
    # plot the roc curve for the model
    pyplot.plot(lr_fpr, lr_tpr, marker='.', label=model_key)
    # axis labels
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    # show the legend
    pyplot.legend()
    # show the plot
pyplot.show()

# Feature Selection

In [150]:
from sklearn.ensemble import RandomForestClassifier

In [151]:
from sklearn.feature_selection import SelectFromModel

In [152]:
model = RandomForestClassifier(n_jobs=-1, random_state=42)

In [153]:
df.head()

,wave_mean,wave_median,wave_std,wave_min,wave_max,wave_amp,wave_SNR,wave_kurtosis,wave_skew,wave_percentile_10,...,fft_SNR,fft_entropy,fft_kurtosis,fft_skew,path,abnormal,id,Machine_pump,Machine_slider,Machine_valve
0,-1.290201,-0.468159,1.289060,0.018168,0.047237,0.01745,-0.753597,-0.258452,-0.134690,-1.282945,...,-0.294127,-0.159342,-0.257215,-0.193474,data/-6_dB_slider/slider/id_04/normal/00000003...,0,04,0,1,0
0,-1.290201,-0.468159,1.289060,0.018168,0.047237,0.01745,-0.753597,-0.258452,-0.134690,-1.282945,...,-0.294127,-0.159342,-0.257215,-0.193474,data/-6_dB_slider/slider/id_04/normal/00000003...,0,04,1,0,0
0,-1.290201,-0.468159,1.289060,0.018168,0.047237,0.01745,-0.753597,-0.258452,-0.134690,-1.282945,...,-0.294127,-0.159342,-0.257215,-0.193474,data/-6_dB_slider/slider/id_04/normal/00000003...,0,04,0,0,0
0,-1.290201,-0.468159,1.289060,0.018168,0.047237,0.01745,-0.753597,-0.258452,-0.134690,-1.282945,...,-0.294127,-0.159342,-0.257215,-0.193474,data/-6_dB_slider/slider/id_04/normal/00000003...,0,04,0,0,1
0,-0.113523,-0.095747,0.581737,0.549018,-0.542826,-0.55657,-0.020836,-0.286158,-0.114634,-0.679514,...,-1.169512,0.231427,0.948522,1.748090,data/0_dB_pump/pump/id_04/normal/00000003.wav,0,04,0,1,0


In [154]:
X = df.drop(['path', 'abnormal'], axis=1)
y = df['abnormal']

In [155]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [156]:
mask_id_6 = x_train['id']=='06'

In [157]:
mask_id_6

907      False
8858      True
9556     False
5187     False
11392    False
         ...  
7691      True
2898     False
11482     True
9807     False
6007     False
Name: id, Length: 173440, dtype: bool

In [158]:
x_train = x_train[~mask_id_6]
y_train = y_train[~mask_id_6]

In [159]:
x_train.head()

,wave_mean,wave_median,wave_std,wave_min,wave_max,wave_amp,wave_SNR,wave_kurtosis,wave_skew,wave_percentile_10,...,fft_max,fft_amp,fft_SNR,fft_entropy,fft_kurtosis,fft_skew,id,Machine_pump,Machine_slider,Machine_valve
907,0.171506,-0.095747,-0.367478,-0.585353,0.141131,0.352660,0.191694,-0.222625,-0.243271,0.348760,...,-0.726259,-0.726256,0.861162,0.697989,-0.504462,-0.775158,00,1,0,0
9556,-0.130216,-0.261263,-1.546534,-1.721173,2.085492,1.955948,-0.646981,5.490667,2.724195,1.622178,...,-1.283315,-1.283317,1.691587,1.479539,-0.497249,-0.665877,00,0,1,0
5187,-0.388146,-0.219884,0.216555,-0.982869,1.422224,1.244186,-0.311059,-0.204689,-0.136968,-0.203646,...,-0.730678,-0.730677,2.077198,1.110519,-0.551950,-1.037379,04,1,0,0
11392,-0.003215,0.111149,0.835060,-0.200259,-0.074948,0.052858,0.083281,-0.254396,-0.204233,-0.865868,...,-0.191262,-0.191255,0.885009,0.573992,-0.466598,-0.661162,00,0,0,1
4446,-2.561836,-0.344022,1.118075,-0.879142,2.202726,1.624823,-1.687208,-0.234427,-0.057416,-1.118776,...,-0.326292,-0.326290,-0.983920,-1.280277,-0.486895,-0.564822,04,0,1,0


In [160]:
y_train.head()

907      0
9556     0
5187     1
11392    0
4446     0
Name: abnormal, dtype: int64

In [161]:
mask_id_4 = x_train['id'] == '04'

In [162]:
x_test = x_train[mask_id_4]
x_train = x_train[~mask_id_4]

y_test = y_train[mask_id_4]
y_train = y_train[~mask_id_4]

In [163]:
y_test.shape

(41926,)

In [113]:
x_test.shape

(41911, 195)

In [116]:
model.fit(x_train, y_train)
probs = model.predict_proba(x_test)[:, 1]
auc = roc_auc_score(y_test, probs)
auc_previous = auc

In [117]:
drop_now = []
drop_final = []

model.fit(x_train, y_train)
probs = model.predict_proba(x_test)[:, 1]
auc_previous = roc_auc_score(y_test, probs)

In [118]:
feat_imp = pd.DataFrame({'feat_imp': model.feature_importances_, 'feat': x_train.columns})

In [119]:
feat_imp.set_index('feat_imp', inplace=True)

In [145]:
dropped = []

In [146]:
improve = {}

In [ ]:
for i, feat in enumerate(feat_imp['feat']):
    model.fit(x_train.drop(feat, axis=1), y_train)
    probs = model.predict_proba(x_test.drop(feat, axis=1))[:, 1]
    auc_now = roc_auc_score(y_test, probs)
    if auc_now > auc_previous:
        x_train.drop(feat, axis=1, inplace=True)
        x_test.drop(feat, axis=1, inplace=True)
        dropped.append(feat)
    print(i, feat)
    improve[feat] = auc_now-auc_previous
    print(auc_previous, auc_now, auc_now-auc_previous)
    print('*************\n')
    auc_previous = max(auc_now, auc_previous)

0 wave_mean
0.5272699583103012 0.5347931507993926 0.00752319248909139
*************

1 wave_median
0.5347931507993926 0.5422655229866109 0.007472372187218301
*************

2 wave_std
0.5422655229866109 0.5469112947764057 0.00464577178979475
*************

3 wave_min
0.5469112947764057 0.5367339265024237 -0.010177368273981968
*************

4 wave_max
0.5469112947764057 0.5346596825191452 -0.012251612257260458
*************

5 wave_amp
0.5469112947764057 0.539738256842139 -0.007173037934266602
*************

6 wave_SNR
0.5469112947764057 0.5396075750059022 -0.007303719770503481
*************

7 wave_kurtosis
0.5469112947764057 0.5416864532662 -0.005224841510205636
*************

8 wave_skew
0.5469112947764057 0.536606271732587 -0.01030502304381864
*************

9 wave_percentile_10
0.5469112947764057 0.549861058937993 0.0029497641615873205
*************

10 wave_percentile_20
0.549861058937993 0.5342432202667091 -0.015617838671283857
*************

11 wave_percentile_30
0.549861058937

90 rolloff_std
0.549861058937993 0.5406938014147826 -0.009167257523210348
*************

91 rolloff_min
0.549861058937993 0.5343134764941038 -0.015547582443889207
*************

92 rolloff_max
0.549861058937993 0.5333413565287438 -0.01651970240924916
*************

93 rolloff_amp
0.549861058937993 0.5376207430753698 -0.012240315862623197
*************

94 rolloff_SNR
0.549861058937993 0.5371990110089152 -0.012662047929077769
*************

95 rolloff_entropy
0.549861058937993 0.542848802680438 -0.007012256257554972
*************

96 rolloff_kurtosis
0.549861058937993 0.5301385568009082 -0.01972250213708482
*************

97 rolloff_skew
0.549861058937993 0.5311194071343509 -0.018741651803642068
*************

98 rolloff_mean_0
0.549861058937993 0.5182715509632069 -0.0315895079747861
*************

99 rolloff_std_0
0.549861058937993 0.5388430607322279 -0.0110179982057651
*************

100 zcr_mean
0.549861058937993 0.5289608277586545 -0.02090023117933848
*************

101 zcr_median
0

In [ ]:
feat_imp.sort_index(inplace=True)
feat_imp.set_index('feat_imp', inplace=True)

dropped1 = []
improve1 = {}

In [ ]:
model.fit(x_train.drop(feat, axis=1)[::5], y_train[::5])
probs = model.predict_proba(x_test.drop(feat, axis=1)[::5])[:, 1]
auc_tiny = roc_auc_score(y_test[::5], probs)

In [ ]:
feat_imp = pd.DataFrame({'feat_imp': model.feature_importances_, 'feat': x_train.columns})


In [ ]:
for i, feat in enumerate(feat_imp['feat']):
    model.fit(x_train.drop(feat, axis=1)[::5], y_train[::5])
    probs = model.predict_proba(x_test.drop(feat, axis=1)[::5])[:, 1]
    auc_now = roc_auc_score(y_test[::5], probs)
    if auc_now > auc_tiny:
        dropped1.append(feat)
    print(i, feat)
    improve1[feat] = auc_now-0.5272699583103012
    print(auc_previous, auc_now, auc_now-0.5272699583103012)
    print('*************\n')
    auc_previous = max(auc_now, auc_previous)

In [89]:
a = ['a']
b = ['b']

In [90]:
b.append(a)

In [93]:
b + ['a']

['b', ['a'], 'a']